# Checking the pretraining dataset

- January 23rd
- This document is now outdated, Yepeng created a new set of drug dataset splits that i'll work with. 

In [1]:
import numpy as np
import pandas as pd

unsup = pd.read_csv("DDE/data/unsup_dataset.csv")
sup_train = pd.read_csv("DDE/data/BIOSNAP/sup_train_val.csv")

# Importing Our Data

In [3]:
import os, shutil
import torch
import torch.nn as nn
import numpy as np
import pandas as pd

## Change this as per your directory, this is my symlink
path_base = "/n/data1/hms/dbmi/zitnik/lab/users/yeh803/DDI/processed_data/PrimeKG-DDI/drug_centric_easy_split/"

val_edgelist_between = torch.load(os.path.join(path_base, "val_between_edgelist.pt"))
val_edgelist_within = torch.load(os.path.join(path_base, "val_within_edgelist.pt"))
test_edgelist_between = torch.load(os.path.join(path_base, "test_between_edgelist.pt"))
test_edgelist_within = torch.load(os.path.join(path_base, "test_within_edgelist.pt"))
val_edgelist = (val_edgelist_between, val_edgelist_within)
test_edgelist = (test_edgelist_between, test_edgelist_within)

train_edgelist = torch.load(os.path.join(path_base, "train_edgelist.pt"))
train_edgelist = np.array(train_edgelist)

In [18]:
# path_base = "/n/data1/hms/dbmi/zitnik/lab/users/yeh803/DDI/processed_data/PrimeKG-DDI/drug_centric_easy_split/"
# pretrain_drugs = pd.read_csv(path_base + "/kgstr_pretrain_compound_lookup.csv", index_col=0)
# ddi_drugs = pd.read_csv(path_base + "/kgddi_drug_split.csv", index_col=0)

# ## checking no drug leakage
# assert set(ddi_drugs[ddi_drugs.split=='val'].node_id) & set(pretrain_drugs.node_id) == set()
# assert set(ddi_drugs[ddi_drugs.split=='test'].node_id) & set(pretrain_drugs.node_id) == set()


# pretrain_drugs['split'] = 'pretrain'
# all_drugs = pd.concat([ddi_drugs[['node_id', 'canonical_smiles', 'view_kg', 'split']], pretrain_drugs], axis=0).drop_duplicates(subset='node_id')
# train_drugs = all_drugs[all_drugs.split.isin({'train', 'pretrain'})]  # train drugs here are ddi train drugs & pretrain drugs

# train_drugs.T

drug_index,0,1,2,3,4,5,6,8,9,10,...,6833,6834,6835,6836,6837,6838,6839,6840,6841,6842
node_id,DB00006,DB00007,DB00014,DB00027,DB00035,DB00067,DB00080,DB00093,DB00104,DB00115,...,DB15498,DB15534,DB15569,DB15575,DB15580,DB15624,DB15643,DB15647,DB15665,DB15688
canonical_smiles,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(=O)O)NC(=O)[C@...,CCNC(=O)[C@@H]1CCCN1C(=O)[C@H](CCCNC(=N)N)NC(=...,CC(C)C[C@H](NC(=O)[C@@H](COC(C)(C)C)NC(=O)[C@H...,CC(C)C[C@@H](NC(=O)CNC(=O)[C@@H](NC=O)C(C)C)C(...,N=C(N)NCCC[C@@H](NC(=O)[C@@H]1CCCN1C(=O)[C@@H]...,NC(=O)CCC1NC(=O)C(Cc2ccccc2)NC(=O)C(Cc2ccc(O)c...,CCCCCCCCCC(=O)N[C@@H](Cc1c[nH]c2ccccc12)C(=O)N...,NCCCC[C@H](NC(=O)[C@@H]1CCCN1C(=O)[C@@H]1CSSC[...,C[C@@H](O)[C@@H]1NC(=O)[C@H](CCCCN)NC(=O)[C@@H...,C/C1=C2/N=C(/C=C3\N=C(/C(C)=C4/[C@@H](CCC(N)=O...,...,CCOc1cc(-c2nncn2C)ccc1Nc1ncc2cc(C)nc(NCC(C)(C)...,COc1cc2c(c(OC)c1OC)-c1ccc(O)c(=O)cc1C(NC(C)=O)CC2,C=CC(=O)N1CCN(c2nc(=O)n(-c3c(C)ccnc3C(C)C)c3nc...,CC[C@H]1c2cc3[n-]c(c(CC(=O)OC)c4nc(cc5[n-]c(cc...,CC(C)(C)C(=O)C(Oc1ccc(Cl)cc1)n1ccnc1,Nc1nnc(Sc2ncc([N+](=O)[O-])s2)s1,NCCNCCN1CC1,CC(=O)c1cc(C(=O)N[C@@H](C)Cn2ccc(-c3ccc(C#N)c(...,CNC[C@@H]1OCCc2ccsc21,Cc1cc(C[C@@H](NC(=O)N2CCC(c3cc4ccccc4[nH]c3=O)...
view_kg,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
split,train,train,train,train,train,train,train,train,train,train,...,pretrain,pretrain,pretrain,pretrain,pretrain,pretrain,pretrain,pretrain,pretrain,pretrain


In [71]:
ddi_drugs = pd.read_csv(path_base + "/kgddi_drug_split.csv", index_col=0)
lookup = ddi_drugs.T
ddi_drugs.T[135]

node_id                                                       DB00278
canonical_smiles    CC1CNc2c(cccc2S(=O)(=O)N[C@@H](CCCN=C(N)N)C(=O...
view_kg                                                             1
primekg_ddi                                                         1
split                                                             val
Name: 135, dtype: object

In [83]:
seed_everything(3)

ddi_drugs = pd.read_csv(path_base + "/kgddi_drug_split.csv", index_col=0)
# lookup = ddi_drugs.T

a = ddi_drugs[ddi_drugs["split"] == "train"]
lookup = a.T

In [84]:


train_edgelist = torch.load(os.path.join(path_base, "train_edgelist.pt"))
## Our goal is to have a CSV where:
# DrugID1 #SMILES #DrugID2 #SMILES #Label(pos neg)

assert is_undirected(pos_ddis)
## Positive samples
drug1 = np_train_edgelist[:, 0]
drug2 = np_train_edgelist[:, 1]
d1 = lookup[drug1].T.rename(columns={"drug_index": "Drug1_ID", "canonical_smiles": "Drug1_SMILES"}).reset_index()
d2 = lookup[drug2].T.rename(columns={"drug_index": "Drug2_ID", "canonical_smiles": "Drug2_SMILES"}).reset_index()
train_pos = pd.concat([d1, d2], axis=1, join='inner')
train_pos["label"] = 1

from torch_geometric.utils import is_undirected, to_undirected, to_dense_adj, k_hop_subgraph, coalesce, structured_negative_sampling, structured_negative_sampling_feasible
from torch_geometric.seed import seed_everything

pos_ddis = train_edgelist.T
drug1, _, drug2 = structured_negative_sampling(pos_ddis) # because we set this to be undirected, we only need heads + negatives

neg_d1 = lookup[drug1].T.rename(columns={"drug_index": "Drug1_ID", "canonical_smiles": "Drug1_SMILES"}).reset_index()
neg_d2 = lookup[drug2].T.rename(columns={"drug_index": "Drug2_ID", "canonical_smiles": "Drug2_SMILES"}).reset_index()
train_neg = pd.concat([neg_d1, neg_d2], axis=1, join='inner')
train_neg["label"] = 0

sup_train_df = pd.concat([train_pos, train_neg])


KeyError: '[135, 886, 266, 986, 384, 1521, 697, 502, 418, 928, 111, 2414, 541, 902, 774, 124, 280, 857, 755, 1036, 894, 1779, 2245, 364, 1325, 345, 1213, 107, 408, 416, 2959, 259, 850, 1229, 1918, 184, 68, 712, 560, 204, 200, 58, 628, 282, 889, 63, 2273, 59, 331, 922, 288, 237, 1851, 1933, 2609, 1301, 789, 614, 216, 462, 243, 876, 761, 913, 747, 173, 3046, 56, 706, 116, 1891, 2618, 709, 375, 1012, 1736, 1662, 569, 417, 485, 580, 807, 166, 906, 2630, 1425, 189, 196, 2166, 177, 676, 307, 918, 1807, 186, 525, 1140, 69, 746, 955, 865, 621, 3086, 89, 848, 1408, 2752, 1460, 563, 545, 34, 1501, 228, 192, 234, 591, 714, 271, 1212, 49, 753, 1387, 826, 105, 1447, 1853, 2430, 2127, 1011, 2117, 232, 380, 1735, 805, 43, 481, 130, 287, 601, 893, 1465, 97, 975, 2003, 38, 1020, 86, 1690, 147, 790, 2176, 291, 295, 413, 936, 642, 348, 1608, 1066, 2459, 544, 236, 715, 498, 340, 2524, 2036, 647, 1462, 2116, 1441, 568, 1921, 138, 363, 127, 603, 449, 342, 123, 365, 1833, 1372, 1725, 322, 344, 526, 231, 1051

In [78]:
sup_train_df

,drug_index,node_id,Drug1_SMILES,view_kg,primekg_ddi,split,drug_index,node_id,Drug2_SMILES,view_kg,primekg_ddi,split,label
0,0,DB00006,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(=O)O)NC(=O)[C@...,1,1,train,1716,DB06605,COc1ccc(-n2nc(C(N)=O)c3c2C(=O)N(c2ccc(N4CCCCC4...,1,1,train,1
1,13,DB00120,N[C@@H](Cc1ccccc1)C(=O)O,1,1,train,1716,DB06605,COc1ccc(-n2nc(C(N)=O)c3c2C(=O)N(c2ccc(N4CCCCC4...,1,1,train,1
2,31,DB00159,CC/C=C\C/C=C\C/C=C\C/C=C\C/C=C\CCCC(=O)O,1,1,train,1716,DB06605,COc1ccc(-n2nc(C(N)=O)c3c2C(=O)N(c2ccc(N4CCCCC4...,1,1,train,1
3,33,DB00163,Cc1c(C)c2c(c(C)c1O)CC[C@@](C)(CCC[C@H](C)CCC[C...,1,1,train,1716,DB06605,COc1ccc(-n2nc(C(N)=O)c3c2C(=O)N(c2ccc(N4CCCCC4...,1,1,train,1
4,41,DB00176,COCCCC/C(=N\OCCN)c1ccc(C(F)(F)F)cc1,1,1,train,1716,DB06605,COc1ccc(-n2nc(C(N)=O)c3c2C(=O)N(c2ccc(N4CCCCC4...,1,1,train,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1232621,3004,DB13650,NC(CO)(CO)CO.O=C(O[Al](O)O)[C@H](O)[C@@H](O)[C...,0,1,train,2579,DB12321,CCCCCNC(=O)c1coc([C@@H]2[C@H]3CC[C@H](O3)[C@@H...,0,1,train,0
1232622,3250,DB14513,[MgH2],1,1,train,505,DB00682,CC(=O)CC(c1ccccc1)c1c(O)c2ccccc2oc1=O,1,1,val,0
1232623,3319,DB14845,O=C(Nc1nc2cccc(-c3ccc(CN4CCS(=O)(=O)CC4)cc3)n2...,1,1,train,2386,DB11593,O=c1[nH]c2ccccc2c(=O)o1,0,1,train,0
1232624,3321,DB14895,O=C(Nc1ccc(C[C@@H]2CC[C@H]([C@H](O)c3ccccc3)N2...,1,1,train,1234,DB01605,CC(C)(C)C(=O)OCOC(=O)[C@@H]1N2C(=O)[C@@H](N=CN...,0,1,train,0


True

In [22]:



## Negative Samples
neg_samp = negative_sampling(train_edgelist.T)

drug1 = neg_samp.T[:, 0]
drug2 = neg_samp.T[:, 1]
neg_d1 = lookup[drug1].T.rename(columns={"drug_index": "Drug1_ID", "canonical_smiles": "Drug1_SMILES"}).reset_index()
neg_d2 = lookup[drug2].T.rename(columns={"drug_index": "Drug2_ID", "canonical_smiles": "Drug2_SMILES"}).reset_index()
train_neg = pd.concat([neg_d1, neg_d2], axis=1, join='inner')
train_neg["label"] = 0

## Put it together
sup_train_df = pd.concat([train_pos, train_neg])
sup_train_df.to_csv("our_data/kgddi_sup_train.csv")
sup_train_df

KeyError: '[805, 1265, 789, 2347, 348, 197, 233, 2831, 57, 288, 130, 735, 3022, 418, 633, 634, 109, 877, 89, 893, 316, 815, 650, 1659, 482, 930, 906, 234, 2273, 2583, 485, 576, 408, 848, 651, 184, 50, 727, 556, 959, 2049, 481, 2032, 902, 895, 753, 643, 1493, 522, 319, 875, 911, 168, 837, 2329, 61, 186, 833, 448, 1230, 1254, 646, 451, 1795, 462, 2459, 220, 663, 1405, 250, 373, 706, 874, 766, 863, 1425, 3301, 752, 364, 1095, 573, 1642, 601, 1460, 619, 436, 761, 279, 696, 525, 43, 287, 550, 232, 541, 27, 751, 173, 644, 2946, 517, 928, 340, 1465, 545, 1541, 291, 1408, 955, 842, 1108, 1533, 1690, 282, 111, 2181, 147, 1016, 59, 65, 603, 280, 840, 829, 767, 2414, 216, 503, 333, 1577, 82, 140, 1279, 647, 1391, 417, 437, 590, 77, 1921, 1011, 297, 679, 469, 342, 1057, 1023, 852, 424, 762, 375, 997, 1441, 193, 330, 505, 886, 2618, 876, 741, 986, 998, 428, 169, 294, 894, 676, 122, 2679, 820, 376, 467, 107, 2081, 393, 621, 494, 2176, 63, 1713, 3205, 568, 660, 1909, 870, 975, 1229, 1430, 177, 488, 5

In [7]:
####
# Quick Sanity checks
###

## Verifying the dataset split
casterdb = sup_train[sup_train["Drug1_ID"] == "DB00706"].iloc[0]["Drug1_SMILES"]
kgddidb = kgddi_drug_lookup[kgddi_drug_lookup["node_id"] == "DB00706"].iloc[0]["canonical_smiles"]

print(casterdb)
print(kgddidb)

assert kgddidb == casterdb


## Positive samples for Val Between
drug1 = val_edgelist_between[:, 0]
drug2 = val_edgelist_between[:, 1]

set(drug1.tolist()) & set(drug2.tolist())

CCOc1ccccc1OCCN[C@H](C)Cc1ccc(OC)c(S(N)(=O)=O)c1
CCOc1ccccc1OCCN[C@H](C)Cc1ccc(OC)c(S(N)(=O)=O)c1


# Validation (between + within) drug pairs for supervised

In [36]:
val_edgelist_bt = torch.load(path_base + '/val_between_edgelist.pt')
val_drugs = val_edgelist_bt[:, 0]
train_drugs = val_edgelist_bt[:, 1]

from itertools import product
all_train_val_pairs = list(product(torch.unique(val_drugs).tolist(), torch.unique(train_drugs).tolist()))
len(all_train_val_pairs)

temp = {pair:0 for pair in all_train_val_pairs}
for tup in val_edgelist_bt.tolist():
    temp[tuple(tup)] = 1
    
temp = pd.DataFrame([(k[0], k[1], v) for k, v in temp.items()])

d1 = lookup[list(temp[0])].T.reset_index().rename(columns={"drug_index": "Drug1_ID", "canonical_smiles": "Drug1_SMILES"})
d2 = lookup[list(temp[1])].T.reset_index().rename(columns={"drug_index": "Drug2_ID", "canonical_smiles": "Drug2_SMILES"})

val_bt = pd.concat([d1, d2, temp], axis=1, join='inner').rename(columns={2: "label"})
# val_bt = []

val_bt = val_bt[["Drug1_ID", "Drug1_SMILES", "Drug2_ID", "Drug2_SMILES", "label"]]
val_bt.to_csv("kgddi_sup_val_between.csv")

In [37]:
val_bt

,Drug1_ID,Drug1_SMILES,Drug2_ID,Drug2_SMILES,label
0,27,Cc1ncc(C[n+]2csc(CCO)c2C)c(N)n1,0,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(=O)O)NC(=O)[C@...,0
1,27,Cc1ncc(C[n+]2csc(CCO)c2C)c(N)n1,1,CCNC(=O)[C@@H]1CCCN1C(=O)[C@H](CCCNC(=N)N)NC(=...,0
2,27,Cc1ncc(C[n+]2csc(CCO)c2C)c(N)n1,2,CC(C)C[C@H](NC(=O)[C@@H](COC(C)(C)C)NC(=O)[C@H...,0
3,27,Cc1ncc(C[n+]2csc(CCO)c2C)c(N)n1,3,CC(C)C[C@@H](NC(=O)CNC(=O)[C@@H](NC=O)C(C)C)C(...,0
4,27,Cc1ncc(C[n+]2csc(CCO)c2C)c(N)n1,4,N=C(N)NCCC[C@@H](NC(=O)[C@@H]1CCCN1C(=O)[C@@H]...,0
...,...,...,...,...,...
1008795,3249,C[C@@H]1C[C@H]2[C@@H]3CCC4=CC(=O)C=C[C@]4(C)[C...,3360,CC(=O)OCC(=O)[C@@]1(O)CC[C@H]2[C@@H]3CCC4=CC(=...,0
1008796,3249,C[C@@H]1C[C@H]2[C@@H]3CCC4=CC(=O)C=C[C@]4(C)[C...,3361,Cc1occc(=O)c1[O-].Cc1occc(=O)c1[O-].Cc1occc(=O...,0
1008797,3249,C[C@@H]1C[C@H]2[C@@H]3CCC4=CC(=O)C=C[C@]4(C)[C...,3363,CC(C)CN(C[C@@H](O)[C@H](Cc1ccccc1)NC(=O)O[C@H]...,0
1008798,3249,C[C@@H]1C[C@H]2[C@@H]3CCC4=CC(=O)C=C[C@]4(C)[C...,3364,C=C1NC(=O)c2nc(oc2C)C(=C)NC(=O)[C@H](C(C)(C)O)...,0


In [34]:
# temp.rename(columns={0: "Drug1_ID", "canonical_smiles": "Drug1_SMILES"})

# 272128 len(val_bt)
# np.sum(val_bt["label"])


In [98]:
# ##
# # Val between
# ##



# ## Positive samples for Val Between
# drug1 = val_edgelist_between[:, 0]
# drug2 = val_edgelist_between[:, 1]
# d1 = lookup[drug1].T.rename(columns={"drug_index": "Drug1_ID", "canonical_smiles": "Drug1_SMILES"}).reset_index()
# d2 = lookup[drug2].T.rename(columns={"drug_index": "Drug2_ID", "canonical_smiles": "Drug2_SMILES"}).reset_index()
# val_between_pos = pd.concat([d1, d2], axis=1, join='inner')
# val_between_pos["label"] = 1

# ## Negative Samples
# val_between_neg_samp = negative_sampling(val_edgelist_between.T)

# drug1 = val_between_neg_samp.T[:, 0]
# drug2 = val_between_neg_samp.T[:, 1]
# neg_d1 = lookup[drug1].T.rename(columns={"drug_index": "Drug1_ID", "canonical_smiles": "Drug1_SMILES"}).reset_index()
# neg_d2 = lookup[drug2].T.rename(columns={"drug_index": "Drug2_ID", "canonical_smiles": "Drug2_SMILES"}).reset_index()
# val_between_neg = pd.concat([neg_d1, neg_d2], axis=1, join='inner')
# val_between_neg["label"] = 0

# ## Put it together
# sup_val_between_df = pd.concat([val_between_pos, val_between_neg])
# sup_val_between_df.to_csv("kgddi_sup_val_between.csv")
# sup_val_between_df

,drug_index,Drug1_SMILES,drug_index,Drug2_SMILES,label
0,27,Cc1ncc(C[n+]2csc(CCO)c2C)c(N)n1,47,CN1CCC[C@H]1c1cccnc1,1
1,27,Cc1ncc(C[n+]2csc(CCO)c2C)c(N)n1,66,CN(CCOc1ccc(NS(C)(=O)=O)cc1)CCc1ccc(NS(C)(=O)=...,1
2,27,Cc1ncc(C[n+]2csc(CCO)c2C)c(N)n1,174,COc1ccc2c3c1O[C@H]1[C@@H](O)C=C[C@H]4[C@@H](C2...,1
3,27,Cc1ncc(C[n+]2csc(CCO)c2C)c(N)n1,221,NC[C@H](O)c1ccc(O)c(O)c1,1
4,27,Cc1ncc(C[n+]2csc(CCO)c2C)c(N)n1,312,C=C[C@H]1CN2CC[C@H]1C[C@H]2[C@H](O)c1ccnc2ccc(...,1
...,...,...,...,...,...
272123,2604,Cc1cc(F)ccc1[C@H]1C[C@@H](N2CCN3C(=O)CC[C@H]3C...,365,CCC1(c2ccccc2)NC(=O)N(C)C1=O,0
272124,2505,CCN1CCN(Cc2ccc(Nc3ncc(F)c(-c4cc(F)c5nc(C)n(C(C...,1891,Cc1c(O)c(=O)ccn1C,0
272125,3097,S=c1nc[nH]c2[nH]ncc12,2597,O=C(CCCN1CCC(O)(c2ccc(Br)cc2)CC1)c1ccc(F)cc1,0
272126,1448,CC1(C)Cc2c(-c3ccccc3)c(-c3ccc(Cl)cc3)c(CC(=O)O...,353,CCCC(CCC)C(=O)OC1CC2CCC(C1)[N+]2(C)C.[Br-],0


In [35]:
# ##
# # Val Within
# ##

val_edgelist_wt = torch.load(path_base + '/val_within_edgelist.pt')
val_drugs1 = val_edgelist_wt[:, 0]
val_drugs2 = val_edgelist_wt[:, 1]

from itertools import product
all_val_val_pairs = list(product(torch.unique(val_drugs1).tolist(), torch.unique(val_drugs2).tolist()))
len(all_val_val_pairs)

temp = {pair:0 for pair in all_val_val_pairs}
for tup in val_edgelist_wt.tolist():
    temp[tuple(tup)] = 1
    
temp = pd.DataFrame([(k[0], k[1], v) for k, v in temp.items()])

d1 = lookup[list(temp[0])].T.reset_index().rename(columns={"drug_index": "Drug1_ID", "canonical_smiles": "Drug1_SMILES"})
d2 = lookup[list(temp[1])].T.reset_index().rename(columns={"drug_index": "Drug2_ID", "canonical_smiles": "Drug2_SMILES"})

val_wt = pd.concat([d1, d2, temp], axis=1, join='inner').rename(columns={2: "label"})

val_wt = val_wt[["Drug1_ID", "Drug1_SMILES", "Drug2_ID", "Drug2_SMILES", "label"]]
val_wt.to_csv("kgddi_sup_val_within.csv")
val_wt


,Drug1_ID,Drug1_SMILES,Drug2_ID,Drug2_SMILES,label
0,27,Cc1ncc(C[n+]2csc(CCO)c2C)c(N)n1,27,Cc1ncc(C[n+]2csc(CCO)c2C)c(N)n1,0
1,27,Cc1ncc(C[n+]2csc(CCO)c2C)c(N)n1,34,Cc1ncc(CO)c(CO)c1O,0
2,27,Cc1ncc(C[n+]2csc(CCO)c2C)c(N)n1,38,CC1=CC(=O)c2ccccc2C1=O,0
3,27,Cc1ncc(C[n+]2csc(CCO)c2C)c(N)n1,40,CC[C@H](C)C(=O)O[C@H]1C[C@H](O)C=C2C=C[C@H](C)...,0
4,27,Cc1ncc(C[n+]2csc(CCO)c2C)c(N)n1,43,CCOC(=O)[C@H](CCc1ccccc1)N[C@@H](C)C(=O)N1[C@H...,0
...,...,...,...,...,...
149764,3249,C[C@@H]1C[C@H]2[C@@H]3CCC4=CC(=O)C=C[C@]4(C)[C...,2967,CCN(CC)CCOC(=O)C(Cc1cccc2ccccc12)CC1CCCO1,0
149765,3249,C[C@@H]1C[C@H]2[C@@H]3CCC4=CC(=O)C=C[C@]4(C)[C...,3022,CN(C)CC[C@@H](c1ccc(Cl)cc1)c1ccccn1,0
149766,3249,C[C@@H]1C[C@H]2[C@@H]3CCC4=CC(=O)C=C[C@]4(C)[C...,3073,CN1CCN(c2c(F)cc3c(=O)c(C(=O)O)cn4c3c2SCC4)CC1,1
149767,3249,C[C@@H]1C[C@H]2[C@@H]3CCC4=CC(=O)C=C[C@]4(C)[C...,3205,COc1ccc(CN2CCN(C(c3ccc(F)cc3)c3ccc(F)cc3)CC2)c...,0


In [99]:
# ##
# # Val Within
# ##


# ## Positive samples for Val within
# drug1 = val_edgelist_within[:, 0]
# drug2 = val_edgelist_within[:, 1]
# d1 = lookup[drug1].T.rename(columns={"drug_index": "Drug1_ID", "canonical_smiles": "Drug1_SMILES"}).reset_index()
# d2 = lookup[drug2].T.rename(columns={"drug_index": "Drug2_ID", "canonical_smiles": "Drug2_SMILES"}).reset_index()
# val_within_pos = pd.concat([d1, d2], axis=1, join='inner')
# val_within_pos["label"] = 1


# ## Negative Samples
# val_within_neg_samp = negative_sampling(val_edgelist_within.T)

# drug1 = val_within_neg_samp.T[:, 0]
# drug2 = val_within_neg_samp.T[:, 1]
# neg_d1 = lookup[drug1].T.rename(columns={"drug_index": "Drug1_ID", "canonical_smiles": "Drug1_SMILES"}).reset_index()
# neg_d2 = lookup[drug2].T.rename(columns={"drug_index": "Drug2_ID", "canonical_smiles": "Drug2_SMILES"}).reset_index()
# val_within_neg = pd.concat([neg_d1, neg_d2], axis=1, join='inner')
# val_within_neg["label"] = 0

# ## Put it together
# sup_val_within_df = pd.concat([val_within_pos, val_within_neg])
# sup_val_within_df.to_csv("kgddi_sup_val_within.csv")
# sup_val_within_df

,drug_index,Drug1_SMILES,drug_index,Drug2_SMILES,label
0,40,CC[C@H](C)C(=O)O[C@H]1C[C@H](O)C=C2C=C[C@H](C)...,1325,C[C@H](CCC(=O)O)[C@H]1CC[C@H]2[C@H]3[C@H](C[C@...,1
1,61,CC[C@H]1OC(=O)[C@H](C)[C@@H](O[C@H]2C[C@@](C)(...,1325,C[C@H](CCC(=O)O)[C@H]1CC[C@H]2[C@H]3[C@H](C[C@...,1
2,68,COC(=O)[C@H]1[C@H]2C[C@@H]3c4[nH]c5cc(OC)ccc5c...,1325,C[C@H](CCC(=O)O)[C@H]1CC[C@H]2[C@H]3[C@H](C[C@...,1
3,76,Cc1cccc(Nc2ccncc2S(=O)(=O)NC(=O)NC(C)C)c1,1325,C[C@H](CCC(=O)O)[C@H]1CC[C@H]2[C@H]3[C@H](C[C@...,1
4,82,Cc1c(O)cccc1C(=O)N[C@@H](CSc1ccccc1)[C@H](O)CN...,1325,C[C@H](CCC(=O)O)[C@H]1CC[C@H]2[C@H]3[C@H](C[C@...,1
...,...,...,...,...,...
60845,687,CCOC(=O)[C@H](CCc1ccccc1)N[C@@H](C)C(=O)N1Cc2c...,1720,C[C@H](NC(=O)C(C)(C)Oc1ccc(C(F)(F)F)cn1)[C@@H]...,0
60846,688,CCN(CC)CCOc1ccc(C(=C(Cl)c2ccccc2)c2ccccc2)cc1,2343,CC[C@H]1OC(=O)C[C@@H](O)[C@H](C)[C@@H](O[C@@H]...,0
60847,1132,CCC(=O)O[C@H]1CC[C@H]2[C@@H]3CCC4=CC(=O)CC[C@]...,1296,CCSC(=N)N,0
60848,1997,O=C(CCCl)NCc1ccccc1,2771,C[C@@H]1C[C@H]2[C@@H]3C[C@H](F)C4=CC(=O)C=C[C@...,0
